In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats#all_results2024-202591"

In [6]:
data = requests.get(standings_url)

In [8]:
soup = BeautifulSoup(data.text, 'html.parser')

In [10]:
standings_table = soup.select('table.stats_table')[0]

In [12]:
links = standings_table.find_all('a')

In [14]:
links = [l.get("href") for l in links]

In [16]:
links = [l for l in links if'/squads/' in l]

In [18]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [20]:
team_url = team_urls[0]

In [22]:
data = requests.get(team_urls[0])

In [23]:
matches = pd.read_html(data.text, match="Scores & Fixtures")

C:\Users\muham\AppData\Local\Temp\ipykernel_21064\1970929637.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")


In [26]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [28]:
data = requests.get(f"https://fbref.com{links[0]}")

In [30]:
shooting = pd.read_html(data.text, match="Shooting")[0]

C:\Users\muham\AppData\Local\Temp\ipykernel_21064\2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [32]:
shooting.columns = shooting.columns.droplevel()

In [34]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,14.8,0.0,0,0,2.6,2.6,0.15,-0.6,-0.6,Match Report
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,13.6,1.0,0,0,2.5,2.5,0.14,-0.5,-0.5,Match Report
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,13.4,0.0,0,0,1.8,1.8,0.16,1.2,1.2,Match Report
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,14.9,0.0,0,0,0.9,0.9,0.07,-0.9,-0.9,Match Report
4,2024-09-17,21:00,Champions Lg,League phase,Tue,Away,W,3.0,1.0,it Milan,...,15.7,1.0,0,0,3.1,3.1,0.14,-0.1,-0.1,Match Report


In [36]:
team_data = matches[0].merge(shooting[["Date" ,"Sh" , "SoT" , "Dist" , "FK" , "PK" , "PKatt"]] , on="Date")

In [38]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,4-2-3-1,Tim Robinson,Match Report,NaN,18,5,14.8,0.0,0,0
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,4-4-2,Stuart Attwell,Match Report,NaN,19,8,13.6,1.0,0,0
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,3,13.4,0.0,0,0
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,4-2-3-1,Michael Oliver,Match Report,NaN,14,5,14.9,0.0,0,0
4,2024-09-17,21:00,Champions Lg,League phase,Tue,Away,W,3.0,1.0,it Milan,...,4-2-3-1,Espen Eskås,Match Report,NaN,23,11,15.7,1.0,0,0


In [40]:
years = list(range(2024, 2020, -1))

In [42]:
years

[2024, 2023, 2022, 2021]

In [44]:
all_matches = []

In [46]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [50]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")

        # Get matches data
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        matches["Date"] = pd.to_datetime(matches["Date"])  # Ensure datetime format

        # Get shooting data
        soup = BeautifulSoup(data.text, 'html.parser')
        links = [l.get("href") for l in soup.find_all('a')]
        shooting_links = [l for l in links if l and 'all_comps/shooting/' in l]
        if not shooting_links:
            print(f"Shooting data not found for {team_name} in {year}")
            continue

        shooting_url = f"https://fbref.com{shooting_links[0]}"
        data = requests.get(shooting_url)
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        shooting["Date"] = pd.to_datetime(shooting["Date"])  # Ensure datetime format

        try:
            # Merge matches and shooting data
            team_data = matches.merge(
                shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]],
                on="Date",
            )
        except ValueError:
            print(f"Merge failed for {team_name} in {year}")
            continue

        # Filter for Premier League matches
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)

        time.sleep(5)  # Avoid overwhelming the server

C:\Users\muham\AppData\Local\Temp\ipykernel_21064\2630105720.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\muham\AppData\Local\Temp\ipykernel_21064\2630105720.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\muham\AppData\Local\Temp\ipykernel_21064\2630105720.py:19: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\muham\AppData\Local\Temp\ipykernel_21064\2630105720.py:32: FutureWarning: Passing literal 

In [52]:
len(all_matches)

81

In [54]:
match_df = pd.concat(all_matches)

In [56]:
match_df.columns = [c.lower() for c in match_df.columns]

In [58]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,Match Report,NaN,18.0,5.0,14.8,0.0,0,0,2024,Liverpool
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,Match Report,NaN,19.0,8.0,13.6,1.0,0,0,2024,Liverpool
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,Match Report,NaN,11.0,3.0,13.4,0.0,0,0,2024,Liverpool
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,Match Report,NaN,14.0,5.0,14.9,0.0,0,0,2024,Liverpool
5,2024-09-21,15:00,Premier League,Matchweek 5,Sat,Home,W,3.0,0.0,Bournemouth,...,Match Report,NaN,19.0,12.0,16.6,0.0,0,0,2024,Liverpool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,18.2,0.0,0,0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,13.4,1.0,0,0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,18.5,0.0,0,0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,18.3,1.0,0,0,2021,Sheffield United


In [60]:
match_df.to_csv("matches.csv")

In [62]:
match_df.shape

(3053, 28)